In [5]:
pip install tables

     |████████████████████████████████| 4.4MB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 194kB 16.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import psutil
import sys
from multiprocessing import Pool
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.layers import Convolution1D, MaxPooling1D, AveragePooling1D
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import *
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import h5py
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [16]:
#need to change to your datapath
data = pd.read_hdf('random_forest.h5')

In [17]:
data

,zip5,impact_score,x,y,lat,lng,5_Wave_Geopotential_Height_isobaric,Absolute_vorticity_isobaric,Apparent_temperature_height_above_ground,Best_4_layer_Lifted_Index_surface,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,2722,20.268081,578.0,97.0,41.5,-71.0,5493.184570,0.000099,275.700012,11.800000,...,0,0,0,0,0,0,0,0,0,0
1,2722,20.268081,578.0,97.0,41.5,-71.0,5514.240723,0.000094,275.399994,10.700000,...,0,0,0,0,0,0,0,0,0,0
2,2722,20.268081,578.0,97.0,41.5,-71.0,5544.382324,0.000093,274.299988,11.100000,...,0,0,0,0,0,0,0,0,0,0
3,2722,20.268081,578.0,97.0,41.5,-71.0,5575.850586,0.000093,275.700012,19.100000,...,0,0,0,0,0,0,0,0,0,0
4,2722,16.868994,578.0,97.0,41.5,-71.0,5614.513184,0.000086,272.899994,22.900000,...,0,0,0,0,0,0,0,0,0,0
5,2722,16.868994,578.0,97.0,41.5,-71.0,5639.217285,0.000091,271.899994,20.200001,...,0,0,0,0,0,0,0,0,0,0
6,2722,16.868994,578.0,97.0,41.5,-71.0,5657.213867,0.000091,271.000000,19.799999,...,0,0,0,0,0,0,0,0,0,0
7,2722,16.868994,578.0,97.0,41.5,-71.0,5659.080078,0.000095,276.200012,21.900000,...,0,0,0,0,0,0,0,0,0,0
8,2722,19.150996,578.0,97.0,41.5,-71.0,5654.876465,0.000092,274.100006,14.200000,...,0,0,0,0,0,0,0,0,0,0
9,2722,19.150996,578.0,97.0,41.5,-71.0,5632.398926,0.000095,274.399994,6.400000,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#reshaping functions Chitwan made
def rolling_window(a, window):
    a_copy = a.copy()
    shape = a_copy.shape[:-1] + (a_copy.shape[-1] - window + 1, window)
    strides = a_copy.strides + (a_copy.strides[-1],)
    return np.lib.stride_tricks.as_strided(a_copy, shape=shape, strides=strides)

def reshape_X(X, time_interval):
    XT = np.transpose(X.values)
    reshaped_X = np.transpose(rolling_window(XT, time_interval), (1,2,0))
    return reshaped_X

In [19]:
# list of zips
zips = list(data.zip5.unique())
zips1 = zips[:23]
zips2 = zips[23:46]
zips3 = zips[46:69]
zips4 = zips[69:]
zipsx = [29172, 2722]

In [28]:
#dictionary of predictions
predictions = {}

#list of rsme
rsme = []


#change zips to your set of zips
for z in zipsx:
    # Save model path
    checkpointer_path = f"./models/{z}.hdf5"
    
    #subsetting data for model
    test = data[data['zip5'] == z]
    test = test.drop(['zip5', 'datetime'], axis =1)
    test_17_18 = test[test['year_2019'] != 1]
    
    #19 test
    test_19 = test[test['year_2019'] == 1]
    test_19 = test_17_18[-27:].append(test_19, ignore_index = True)
    test_19 = test_19.drop(['impact_score'], axis = 1)
    test_19 = reshape_X(test_19, 28)
    
    #final y list
    y = test_17_18['impact_score']
    y = y[27:]
    
    #final X
    X = test_17_18.drop(['impact_score'], axis = 1)
    X = reshape_X(X, 28)
    
    
    #data split
    test_size = int(0.3 * X.shape[0])
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
    
    checkpointer = ModelCheckpoint(checkpointer_path, monitor='val_loss', verbose=1, save_best_only=True)
    
    #model architecture
    model = Sequential((
        Convolution1D(input_shape=(28, 173), 
                      kernel_size=4, activation="relu", filters=4),
        MaxPooling1D(),    
        Dropout(0.2),
        Convolution1D(kernel_size=4, activation="relu", filters=4),
        MaxPooling1D(),
        Dropout(0.2),
        Convolution1D(kernel_size=4, activation="relu", filters=4),
        Flatten(),
        Dense(1, activation='linear'),
        Dense(1, activation='linear'),
    ))
    opt = Adam(lr=0.0001)
    
    #output of model while running
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])
    model.summary()
    
    #stop to prevent overfitting
    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
    
    #model fit and test
    val = model.fit(X_train, y_train, epochs=150, batch_size=5, validation_data=(X_test, y_test), callbacks=[checkpointer])
    pred = model.predict(X_test)
    testScore = math.sqrt(mean_squared_error(y_test,pred))
    rsme.append(testScore)
    
    #model predictions
    model = load_model(f"./models/{z}.hdf5")
    pred_19 = model.predict(test_19)
    predictions[z] = pred_19

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_31 (Conv1D)           (None, 25, 4)             2772      
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 12, 4)             0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 12, 4)             0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 9, 4)              68        
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 4, 4)              0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 4, 4)              0         
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 1, 4)              68        
__________


Epoch 00024: val_loss improved from 271.45175 to 270.43717, saving model to ./models/29172.hdf5
Epoch 25/150
2026/2026 [==============================] - 2s 1ms/step - loss: 313.3588 - mean_squared_error: 313.3588 - val_loss: 269.2349 - val_mean_squared_error: 269.2349

Epoch 00025: val_loss improved from 270.43717 to 269.23487, saving model to ./models/29172.hdf5
Epoch 26/150
2026/2026 [==============================] - 2s 1ms/step - loss: 311.9564 - mean_squared_error: 311.9564 - val_loss: 267.8260 - val_mean_squared_error: 267.8260

Epoch 00026: val_loss improved from 269.23487 to 267.82604, saving model to ./models/29172.hdf5
Epoch 27/150
2026/2026 [==============================] - 2s 1ms/step - loss: 310.3258 - mean_squared_error: 310.3258 - val_loss: 266.1979 - val_mean_squared_error: 266.1979

Epoch 00027: val_loss improved from 267.82604 to 266.19793, saving model to ./models/29172.hdf5
Epoch 28/150
2026/2026 [==============================] - 2s 1ms/step - loss: 308.4589 - m

2026/2026 [==============================] - 2s 1ms/step - loss: 232.3763 - mean_squared_error: 232.3763 - val_loss: 193.5821 - val_mean_squared_error: 193.5821

Epoch 00055: val_loss improved from 196.19644 to 193.58212, saving model to ./models/29172.hdf5
Epoch 56/150
2026/2026 [==============================] - 2s 1ms/step - loss: 229.5255 - mean_squared_error: 229.5255 - val_loss: 190.9573 - val_mean_squared_error: 190.9573

Epoch 00056: val_loss improved from 193.58212 to 190.95727, saving model to ./models/29172.hdf5
Epoch 57/150
2026/2026 [==============================] - 2s 1ms/step - loss: 226.6568 - mean_squared_error: 226.6568 - val_loss: 188.3130 - val_mean_squared_error: 188.3130

Epoch 00057: val_loss improved from 190.95727 to 188.31295, saving model to ./models/29172.hdf5
Epoch 58/150
2026/2026 [==============================] - 2s 1ms/step - loss: 223.7550 - mean_squared_error: 223.7550 - val_loss: 185.6401 - val_mean_squared_error: 185.6401

Epoch 00058: val_loss imp


Epoch 00085: val_loss did not improve from 2.13183
Epoch 86/150
2026/2026 [==============================] - 2s 1ms/step - loss: 9.3973 - mean_squared_error: 9.3973 - val_loss: 2.6276 - val_mean_squared_error: 2.6276

Epoch 00086: val_loss did not improve from 2.13183
Epoch 87/150
2026/2026 [==============================] - 2s 1ms/step - loss: 9.1588 - mean_squared_error: 9.1588 - val_loss: 2.8066 - val_mean_squared_error: 2.8066

Epoch 00087: val_loss did not improve from 2.13183
Epoch 88/150
2026/2026 [==============================] - 2s 1ms/step - loss: 8.2834 - mean_squared_error: 8.2834 - val_loss: 2.7126 - val_mean_squared_error: 2.7126

Epoch 00088: val_loss did not improve from 2.13183
Epoch 89/150
2026/2026 [==============================] - 2s 1ms/step - loss: 9.0993 - mean_squared_error: 9.0993 - val_loss: 2.8071 - val_mean_squared_error: 2.8071

Epoch 00089: val_loss did not improve from 2.13183
Epoch 90/150
2026/2026 [==============================] - 2s 1ms/step - loss

2026/2026 [==============================] - 2s 1ms/step - loss: 7.2583 - mean_squared_error: 7.2583 - val_loss: 2.8832 - val_mean_squared_error: 2.8832

Epoch 00123: val_loss did not improve from 2.13183
Epoch 124/150
2026/2026 [==============================] - 2s 1ms/step - loss: 7.4847 - mean_squared_error: 7.4847 - val_loss: 2.8926 - val_mean_squared_error: 2.8926

Epoch 00124: val_loss did not improve from 2.13183
Epoch 125/150
2026/2026 [==============================] - 2s 1ms/step - loss: 7.4274 - mean_squared_error: 7.4274 - val_loss: 2.9720 - val_mean_squared_error: 2.9720

Epoch 00125: val_loss did not improve from 2.13183
Epoch 126/150
2026/2026 [==============================] - 2s 1ms/step - loss: 7.1220 - mean_squared_error: 7.1220 - val_loss: 2.9737 - val_mean_squared_error: 2.9737

Epoch 00126: val_loss did not improve from 2.13183
Epoch 127/150
2026/2026 [==============================] - 2s 1ms/step - loss: 6.9561 - mean_squared_error: 6.9561 - val_loss: 2.9463 - va

2026/2026 [==============================] - 2s 1ms/step - loss: 14033.6311 - mean_squared_error: 14033.6311 - val_loss: 399.5411 - val_mean_squared_error: 399.5411

Epoch 00002: val_loss improved from 777.88329 to 399.54112, saving model to ./models/2722.hdf5
Epoch 3/150
2026/2026 [==============================] - 2s 1ms/step - loss: 6462.5936 - mean_squared_error: 6462.5936 - val_loss: 258.0932 - val_mean_squared_error: 258.0932

Epoch 00003: val_loss improved from 399.54112 to 258.09322, saving model to ./models/2722.hdf5
Epoch 4/150
2026/2026 [==============================] - 2s 1ms/step - loss: 3583.1372 - mean_squared_error: 3583.1372 - val_loss: 257.7402 - val_mean_squared_error: 257.7402

Epoch 00004: val_loss improved from 258.09322 to 257.74016, saving model to ./models/2722.hdf5
Epoch 5/150
2026/2026 [==============================] - 2s 1ms/step - loss: 3377.3140 - mean_squared_error: 3377.3140 - val_loss: 257.3205 - val_mean_squared_error: 257.3205

Epoch 00005: val_loss

KeyboardInterrupt: 